Progress:

DONE:
-get package to work from command line
-generate output from fasta files
-programatically get package to work on one sequence
-figure out indexing/confirm that package runs through dataframe in order

NOT DONE:
-send outfiles files into dataframe
-get command line argument incorporated into script
-scale beyond 5 sequences
-train model with embeddings and compare results

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest

In [2]:
import torch
import esm

In [3]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [4]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k_exploration.csv').sample(200,random_state=42)

In [5]:
df

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
33553,33553,0.303922,0.276786,0.216783,199,0.888393,211,0.606322,221,9827,...,1004.0,26.0,50.0,24.0,MTNCKVDNKGWINVSDGSQLLDIQHLHTGFRIKDEYYDAVDDVSFE...,MDIKLHHVNKSYETGVETINVLKDIDLTIESGSWLTITGPSGSGKT...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,348,224
9427,9427,0.303371,0.173633,0.190476,178,0.572347,196,0.765625,70,875,...,1035.0,30.0,50.0,20.0,MRNVAALSMPPRVPVEEGRTFTVDEGKRLLDAVADHRMGVLVLLAL...,MSGTRPDRVIRAFVDHLVVERGMARNTVDSYARDLRRYHAHLQARG...,site-specific integrase,site-specific tyrosine recombinase XerD,256,311
199,199,0.292453,0.185629,0.184799,213,0.637725,215,0.637982,140,11324,...,989.0,32.0,54.0,22.0,MFTLKHATAASAVVLGLVFATETFAADVTWNVSLWGKRRAFTEHVE...,MKIAREFAGALAGTLAGFAMAAVTALPVFAEELKFANFTPPFHTIN...,C4-dicarboxylate TRAP transporter substrate-bi...,TRAP transporter substrate-binding protein,337,334
12447,12447,0.406250,0.369919,0.366936,221,0.898374,230,0.920000,315,14963,...,1081.0,28.0,52.5,24.5,MVKADRVHKNFGALKVLKGISLEVGRGEVLCLVGPSGSGKSTFLRC...,MPPVLEARDIAKSFGPTPALRGLSLTIDAGEVVAITGPSGSGKSTL...,amino acid ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,250,246
39489,39489,0.412371,0.207254,0.172414,111,0.575129,102,0.376384,47,12872,...,1044.0,21.5,45.0,23.5,MIISLPAALGAAVLFGVASALQAYAARTEPPGSGLRGLLRVVLRLP...,MISLTVTAVVFGVVFLAELPDKTALAGLVLGTRYRASYVFAGVAAA...,hypothetical protein,TMEM165/GDT1 family protein,271,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37891,37891,0.279070,0.255319,0.257143,258,0.914894,256,0.920863,251,14963,...,1179.0,31.0,52.5,21.5,MVKRLNFMGVIARVFMWSFLLGLAVIVLYPLLWMALNGFKTNAELF...,MATPQAGARRYGPGRYAVLTAYLVLAVMMLVPFAIVLLNAIKTPRE...,carbohydrate ABC transporter permease,carbohydrate ABC transporter permease,278,282
26178,26178,0.280000,0.230047,0.208511,171,0.802817,189,0.735409,115,6260,...,994.0,25.0,55.0,30.0,MSDNIRLSVKGVNKRFGGLQALSDVGLEIKSGQIYGLIGPNGAGKT...,MLEASALTCIKGDRILFRDLTHVFRPGVLTRIAGPNGVGKTSLLRL...,ABC transporter ATP-binding protein,cytochrome c biogenesis heme-transporting ATPa...,257,213
45892,45892,0.368000,0.212963,0.216981,142,0.657407,131,0.629808,61,14963,...,1267.0,30.0,52.5,22.5,MSRADPPAPPALKPGRGRRPAEDVRRAVLAAAAELLLAEGMPGMTF...,MTSRAPSREPQQDRSRATRRRLLEAAIDCLASVGWAGTTVAVVAER...,TetR/AcrR family transcriptional regulator C-t...,TetR/AcrR family transcriptional regulator,208,216
20933,20933,0.268775,0.231293,0.235702,255,0.867347,251,0.886926,184,6048,...,1096.0,30.0,50.0,20.0,MVYSEKAAKWVLSFPLLLFWVVMVVFPAGYAIYLSLHDVSMGQGMS...,MTRIVRQAQYLVFLVPILVVYSLFTIYPLLKAFFLSFTNFDGYSKV...,sugar ABC transporter permease,sugar ABC transporter permease,283,294


In [6]:
def get_fasta_from_dataframe(
        dataframe, output_file_a: str, output_file_b: str):
    '''
    Generates fasta file type from pandas dataframe.

    Args:
        Dataframe (pandas dataframe)
        Names of output fasta files (str)

    Returns:
        Two fasta files with protein sequences and pair_id
    '''
    # meso sequence to fasta
    with open(f'{output_file_a}', 'w') as f:
        for _, row in dataframe.iterrows():
            f.write(
                '>{}\n{}\n'.format(
                    (row['prot_pair_index']),
                    row['m_protein_seq']))

    # thermo sequence to fasta
    with open(f'{output_file_b}', 'w') as f:
        for _, row in dataframe.iterrows():
            f.write(
                '>{}\n{}\n'.format(
                    (row['prot_pair_index']),
                    (row['t_protein_seq'])))

    # return output files
    return [output_file_a, output_file_b]

In [7]:
get_fasta_from_dataframe(df, 'seq_a.fasta', 'seq_b.fasta')

['seq_a.fasta', 'seq_b.fasta']

In [8]:
import re

Would be a good spot to figure out how to generate esm embeddings from a fasta file.

In [13]:
#this block of code should load the files into a dataframe 

import os

# Create an empty dataframe
embeddings = pd.DataFrame(columns=['prot_pair_index', 'tensor'])

# Set the directory path containing the PyTorch files
directory = '/Users/loganroberts/Learn2Therm/ValidProt/data/pytorch_embeddings'

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.pt'):  # Adjust the file extension as needed
        file_path = os.path.join(directory, filename)
        
        # Load the PyTorch file
        data = torch.load(file_path)
        
        # Extract the tensor(s) from the loaded data
        if isinstance(data, torch.Tensor):
            tensors = [data]
        elif isinstance(data, dict):
            tensors = list(data.values())
        else:
            # Handle other cases based on your specific file structure
            continue
        
        # Append the filename and tensor(s) to the dataframe
        embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)

#expand tensor values into individual columns
embeddings = pd.concat([embeddings.drop('tensor', axis=1),
                embeddings['tensor'].apply(lambda x: pd.Series(x))], axis=1)

<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append me

<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  embeddings = embeddings.append({'prot_pair_index': int(re.sub('.pt', '', filename)), 'tensor': tensors[1][33].numpy()}, ignore_index=True)
<ipython-input-13-2b3af67eb1b2>:29: FutureWarning: The frame.append me

In [14]:
embeddings

,prot_pair_index,0,1,2,3,4,5,6,7,8,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,111336591,0.019533,-0.024063,-0.066794,0.050212,-0.047821,-0.094027,0.005994,-0.042168,-0.076345,...,0.064117,-0.019009,-0.056704,-0.025658,-0.057087,0.068928,0.081193,-0.126052,0.009601,-0.012149
1,94441498,-0.011820,-0.104690,0.038199,-0.035568,-0.113570,0.041435,-0.002200,-0.204471,-0.095335,...,0.065634,-0.101977,-0.032930,0.027585,-0.048157,-0.056852,0.132306,-0.249838,0.005303,0.080105
2,8507781,-0.013917,0.001229,0.015009,0.047818,-0.123596,-0.030345,0.005982,-0.083568,-0.016797,...,-0.024941,-0.100372,0.004829,-0.036445,-0.053202,-0.054582,0.124111,-0.150853,-0.051989,0.058379
3,136710423,-0.040277,-0.054379,0.014265,-0.067939,-0.015732,-0.016550,0.022490,-0.081806,-0.056216,...,0.038008,-0.067936,-0.081956,-0.026359,-0.060215,0.091697,0.028884,-0.120096,0.016685,0.030200
4,49002962,-0.027329,-0.096013,-0.018252,-0.009544,-0.085416,-0.035731,0.041572,-0.023275,-0.097101,...,0.049347,-0.068024,-0.072538,-0.035347,-0.035973,0.053484,0.054946,-0.245820,0.033397,0.033206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,140227431,0.000347,-0.010710,0.057421,-0.007362,-0.086283,0.010301,0.083760,-0.098041,-0.090244,...,0.000161,-0.054455,-0.006414,-0.062227,-0.094488,0.007952,0.105958,-0.153057,0.016046,0.086979
196,95379032,-0.060202,-0.008571,0.051682,0.064621,-0.121588,-0.039067,-0.023636,-0.126189,-0.022895,...,-0.037514,-0.084787,0.000140,-0.001242,-0.056960,-0.024581,0.117786,-0.138116,-0.025414,0.024033
197,25255216,-0.032610,-0.030036,-0.021970,0.052059,-0.082433,0.042803,-0.027478,-0.097515,-0.039641,...,-0.040955,-0.044872,-0.015927,-0.078347,-0.040029,-0.021940,0.038414,-0.149151,0.073578,0.089899
198,114520866,-0.015026,-0.027358,0.058169,-0.028993,-0.017491,-0.085501,-0.015761,-0.051320,-0.056967,...,-0.032083,-0.061828,-0.031256,-0.044584,-0.055094,0.062155,0.101345,-0.130985,0.013092,0.049958


In [15]:
new_df = pd.merge(embeddings, df, on='prot_pair_index')

In [16]:
new_df

,prot_pair_index,0,1,2,3,4,5,6,7,8,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,111336591,0.019533,-0.024063,-0.066794,0.050212,-0.047821,-0.094027,0.005994,-0.042168,-0.076345,...,980.0,37.0,57.5,20.5,MGLVTRQKRVAPSIYIRLKTPQKYLRKIEELSSRVIVEPWNWREPE...,MRKFSVVVTEKMLPSISDKLQERCHVREWDRTDPIPRELLLKWLAD...,D-2-hydroxyacid dehydrogenase,D-glycerate dehydrogenase,331,328
1,94441498,-0.011820,-0.104690,0.038199,-0.035568,-0.113570,0.041435,-0.002200,-0.204471,-0.095335,...,1114.0,22.5,46.0,23.5,MTPSRPYIMRALYEWIVDNSCTPYILVDATMEDVSVPEQYVKDGQI...,MTSNRPYILRAFYEWITDNRCTPYILVDTHLPGVLVPQQHVNEDGQ...,ClpXP protease specificity-enhancing factor,ClpXP protease specificity-enhancing factor,133,131
2,8507781,-0.013917,0.001229,0.015009,0.047818,-0.123596,-0.030345,0.005982,-0.083568,-0.016797,...,1082.0,24.0,45.0,21.0,MTTTSETAAEAPRKTLETAVRMEAVNKWYGAFQALKNVDLEVRSGE...,MTLFEDQFAGRHAPSVPPKRSQGGAHAYFSAWNLHAYYGESYIVQG...,amino acid ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,258,259
3,136710423,-0.040277,-0.054379,0.014265,-0.067939,-0.015732,-0.016550,0.022490,-0.081806,-0.056216,...,1077.0,28.0,50.0,22.0,MTTTGSASPYDPPADPAASAAAAAVEQTPPRTDRRVVLVTGASSGI...,MTAEVPANDVEAPVADLSGHRALVTGAASGIGRACAEAFAAAGAEV...,SDR family NAD(P)-dependent oxidoreductase,3-hydroxybutyrate dehydrogenase,279,262
4,49002962,-0.027329,-0.096013,-0.018252,-0.009544,-0.085416,-0.035731,0.041572,-0.023275,-0.097101,...,963.0,30.0,65.0,35.0,MSLTLRQVRYFVATAEIGQISQAAIHLNISQSAVTTAIKELEAMLG...,MADRRLQVFHTVARLLSFTKAAEALHMTQPAVTFQIRQLEEQFNTR...,LysR family transcriptional regulator,selenium metabolism-associated LysR family tra...,305,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,140227431,0.000347,-0.010710,0.057421,-0.007362,-0.086283,0.010301,0.083760,-0.098041,-0.090244,...,945.0,29.0,54.0,25.0,MTAHAKITLKATLLSCAAAATLWSSMAHAQTTVRLGVGDPIGSSVG...,MLKTLTRSLMVATMLAGSAAAGWAADYTLRAVANSNENDEDYDGLV...,DctP family TRAP transporter solute-binding su...,TRAP transporter substrate-binding protein DctP,337,354
196,95379032,-0.060202,-0.008571,0.051682,0.064621,-0.121588,-0.039067,-0.023636,-0.126189,-0.022895,...,958.0,30.0,52.5,22.5,MNAPIISVKDVRKSYGRGPNRFDALKGVSFDIFEGESVAIVGKSGS...,MDDRKQPSGAPEPVLSVRDLVVEYRTGQRPSDPVHRAVDGVSFDLY...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,232,291
197,25255216,-0.032610,-0.030036,-0.021970,0.052059,-0.082433,0.042803,-0.027478,-0.097515,-0.039641,...,1124.0,30.0,52.5,22.5,MTAAVEPAATDAPVGDALVSVEDLKIHFPIKRGLLVDRTVGHVYAV...,MTSTSTAPRTDRAAGTGEPVLSVRDLRVEYHTDAGAVRAVNGVSFD...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,380,341
198,114520866,-0.015026,-0.027358,0.058169,-0.028993,-0.017491,-0.085501,-0.015761,-0.051320,-0.056967,...,1154.0,28.0,50.0,22.0,MSSRKSFVDVARPDSPVALVTGASGGLGQALAIELDALGCRVAVHY...,MSAPGAPLDGKVALITGAARGQGRSHALRLARDGAAIIAVDICAPI...,3-oxoacyl-ACP reductase FabG,mycofactocin-coupled SDR family oxidoreductase,253,283


In [ ]:
python pairpro/extract.py esm2_t33_650M_UR50D notebooks/seq_a.fasta data/pytorch_embeddings/ --include mean

In [17]:
# import subprocess

# # Run the command line prompt and capture the output
# output = subprocess.check_output(['python pairpro/extract.py esm2_t33_650M_UR50D notebooks/seq_a.fasta data/pytorch_embeddings/ --include mean'], universal_newlines=True)

# # Process the captured output as needed
# processed_data = output.strip()

# # Use the processed data in your script
# print("Processed data:", processed_data)


Below code is for running on individual sequences

In [38]:
# # Load ESM-2 model
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# batch_converter = alphabet.get_batch_converter()
# model.eval()  # disables dropout for deterministic results

# # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [(df['prot_pair_index'].iloc[0], df['m_protein_seq'].iloc[0])]

# batch_labels, batch_strs, batch_tokens = batch_converter(data)
# batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# # Extract per-residue representations (on CPU)
# with torch.no_grad():
#     results = model(batch_tokens, repr_layers=[33], return_contacts=True)
# token_representations = results["representations"][33]

# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#     sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# # Look at the unsupervised self-attention map contact predictions
# # import matplotlib.pyplot as plt
# # for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
# #     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
# #     plt.title(seq)
# #     plt.show()

In [18]:
df = new_df

In [19]:
hmmer_target = pd.read_csv('protein_match_50k')
hmmer_target['protein_match'] = hmmer_target['protein_match'].map({'Yes': 1, 'No': 0})
hmmer_target



from sklearn.utils import resample

# Assuming your data is in a pandas DataFrame called 'data'
# Separate the majority and minority classes
majority_class = hmmer_target[hmmer_target['protein_match'] == 1]
minority_class = hmmer_target[hmmer_target['protein_match'] == 0]

# Undersample the majority class to match the number of minority class samples
n_samples = len(minority_class)
undersampled_majority = resample(majority_class, n_samples=n_samples, replace=False)

# Combine the undersampled majority class with the minority class
balanced_data = pd.concat([undersampled_majority, minority_class])

balanced_data

df = pd.merge(df, balanced_data, on=['prot_pair_index'])
df.shape

(67, 1313)

In [20]:
df['protein_match'].value_counts()

1    35
0    32
Name: protein_match, dtype: int64

In [21]:
df['structure_match'] = df['protein_match']

In [22]:
df

,prot_pair_index,0,1,2,3,4,5,6,7,8,...,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len,Unnamed: 0_y,protein_match,Jaccard_Score,structure_match
0,8507781,-0.013917,0.001229,0.015009,0.047818,-0.123596,-0.030345,0.005982,-0.083568,-0.016797,...,MTTTSETAAEAPRKTLETAVRMEAVNKWYGAFQALKNVDLEVRSGE...,MTLFEDQFAGRHAPSVPPKRSQGGAHAYFSAWNLHAYYGESYIVQG...,amino acid ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,258,259,1807,0,0.222222,0
1,9674279,-0.035063,-0.033278,-0.031553,0.050510,-0.125549,-0.023605,-0.023363,-0.054658,-0.000024,...,MVKKSIVLDDSDNYPRIIAKNLEVTYPLGSFSAVKRNRTGFKALDG...,MLDSSEVTVTADGRRIGGVLMDIRNITLRFGGVVAIRDTSFDIREG...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,246,272,2196,1,0.500000,1
2,47032839,-0.042536,-0.001069,0.010911,-0.078783,-0.110655,-0.029389,-0.020749,-0.072975,-0.040882,...,MTLRARLVVARPRFRLDIDLEVASGQVVALLGPNGAGKTTALRALA...,MDLEALTKRYPGQRVPAVENVTMHIPAGEIVVLVGPSGCGKTTTMK...,ABC transporter ATP-binding protein,betaine/proline/choline family ABC transporter...,343,386,1982,0,0.222222,0
3,22228095,-0.048989,-0.084854,-0.017102,-0.018352,-0.079451,-0.022137,0.013828,-0.097690,-0.013558,...,MFYQLLKYIVLGPLLRLVFRPRVEGLEHVPEEGAAIIAGNHLSFAD...,MFYTFLARVVAPILRLLFRPQVEGLENVPAHGPLIIASNHLSFIDS...,1-acyl-sn-glycerol-3-phosphate acyltransferase,1-acyl-sn-glycerol-3-phosphate acyltransferase,242,226,1050,1,1.000000,1
4,8499524,-0.060101,-0.061766,-0.073530,0.034004,-0.124030,-0.007483,-0.046469,-0.068001,-0.033389,...,MSTQADEPRTEGDPQVPVQVGEVVDPRFRHDYLQVRDLRVVFDGFV...,MSDLTLQNVRKVYRTKGRPEVEAVRGMTLTVRSGELVGLLGPSGCG...,urea ABC transporter ATP-binding protein UrtD,ABC transporter ATP-binding protein,286,356,2517,0,0.250000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,95670986,-0.026282,-0.020020,-0.034754,0.026878,-0.132900,-0.105046,0.012186,-0.045722,0.011634,...,MLKVNNVTKFFHKNNKSFCAVDSLDLHISPGEIYAFLGPNGAGKTT...,MIQFINVNKAFGDFQVLKDINLKIEAGEKVVILGPSGSGKSTLLRC...,ABC transporter ATP-binding protein,amino acid ABC transporter ATP-binding protein,303,242,1033,0,0.200000,0
63,141297786,-0.023940,-0.042270,-0.039667,-0.034018,-0.001502,-0.023285,-0.030766,-0.100879,-0.063545,...,MRAVVCPAYGPPEIIRIEERRSPAPGPGQVRVRVGAAAVNFPDVLV...,MPKGVQVVRPGGPEVMSYTDVEVRRPGEGEVLVDVSVAGVNFIDIY...,NADPH:quinone oxidoreductase family protein,quinone oxidoreductase,322,320,803,1,1.000000,1
64,52135150,-0.011473,-0.009117,0.005419,0.016954,-0.113067,-0.036459,-0.026923,-0.072642,-0.044034,...,MSADNTPHGASITLRGLTKTYGSEKAVDAIDLDARPGEFLTLLGPS...,MARVSDAPEVVIDPTREPIVVVDDLHIIYRVYGSAGGGKGTAASAL...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,362,265,1210,0,0.166667,0
65,120473180,-0.066773,-0.072190,-0.065844,0.028404,-0.159087,-0.105056,0.067993,-0.108386,-0.046956,...,MNKKELNWRVSQSEANLSLRAFLREEKQLSKRLLSAIKFQGGEIYL...,MTPTERAYTLTLTVQEGQEGRLAAFLREQHHFSRRLLNKLKREGTV...,RluA family pseudouridine synthase,RluA family pseudouridine synthase,313,299,2597,1,1.000000,1


In [23]:
df_seqs = df[['m_protein_seq', 't_protein_seq', 'prot_pair_index']]
df_seqs

,m_protein_seq,t_protein_seq,prot_pair_index
0,MTTTSETAAEAPRKTLETAVRMEAVNKWYGAFQALKNVDLEVRSGE...,MTLFEDQFAGRHAPSVPPKRSQGGAHAYFSAWNLHAYYGESYIVQG...,8507781
1,MVKKSIVLDDSDNYPRIIAKNLEVTYPLGSFSAVKRNRTGFKALDG...,MLDSSEVTVTADGRRIGGVLMDIRNITLRFGGVVAIRDTSFDIREG...,9674279
2,MTLRARLVVARPRFRLDIDLEVASGQVVALLGPNGAGKTTALRALA...,MDLEALTKRYPGQRVPAVENVTMHIPAGEIVVLVGPSGCGKTTTMK...,47032839
3,MFYQLLKYIVLGPLLRLVFRPRVEGLEHVPEEGAAIIAGNHLSFAD...,MFYTFLARVVAPILRLLFRPQVEGLENVPAHGPLIIASNHLSFIDS...,22228095
4,MSTQADEPRTEGDPQVPVQVGEVVDPRFRHDYLQVRDLRVVFDGFV...,MSDLTLQNVRKVYRTKGRPEVEAVRGMTLTVRSGELVGLLGPSGCG...,8499524
...,...,...,...
62,MLKVNNVTKFFHKNNKSFCAVDSLDLHISPGEIYAFLGPNGAGKTT...,MIQFINVNKAFGDFQVLKDINLKIEAGEKVVILGPSGSGKSTLLRC...,95670986
63,MRAVVCPAYGPPEIIRIEERRSPAPGPGQVRVRVGAAAVNFPDVLV...,MPKGVQVVRPGGPEVMSYTDVEVRRPGEGEVLVDVSVAGVNFIDIY...,141297786
64,MSADNTPHGASITLRGLTKTYGSEKAVDAIDLDARPGEFLTLLGPS...,MARVSDAPEVVIDPTREPIVVVDDLHIIYRVYGSAGGGKGTAASAL...,52135150
65,MNKKELNWRVSQSEANLSLRAFLREEKQLSKRLLSAIKFQGGEIYL...,MTPTERAYTLTLTVQEGQEGRLAAFLREQHHFSRRLLNKLKREGTV...,120473180


In [24]:
#get rid of stuff that isn't quantitative

df = df.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y','thermo_index', 'm_protein_seq', 't_protein_seq',
                        'm_protein_desc', 't_protein_desc'])

In [25]:
df.columns

Index([      'prot_pair_index',                       0,
                             1,                       2,
                             3,                       4,
                             5,                       6,
                             7,                       8,
       ...
       'subject_align_cov_16s',         'bit_score_16s',
                       'm_ogt',                 't_ogt',
              'ogt_difference',         'm_protein_len',
               't_protein_len',         'protein_match',
               'Jaccard_Score',       'structure_match'],
      dtype='object', length=1307)

From pairplot below, it looks like query_align_cov_16s and subject_align_cov_16s dont correlate with target. Let's remove them.

In [26]:
# sns.pairplot(df.sample(500),height = 4)

In [27]:
df = df.drop(columns = ['query_align_cov_16s', 'subject_align_cov_16s'])

In [28]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,scaled_local_symmetric_percent_id_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_len,t_protein_len,protein_match,Jaccard_Score,structure_match
count,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,...,67.000000,67.000000,67.000000,67.00000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,-0.027277,-0.035986,-0.015397,0.019778,-0.075090,-0.019970,0.006330,-0.081295,-0.020644,0.023334,...,0.903224,1056.477612,27.738806,52.00000,24.261194,275.432836,262.373134,0.522388,0.483173,0.522388
std,0.037587,0.034874,0.038562,0.045897,0.056407,0.038783,0.040957,0.042846,0.044200,0.030281,...,0.014996,73.429075,3.478678,3.97816,3.225559,53.030930,56.150187,0.503268,0.344267,0.503268
min,-0.138532,-0.146711,-0.113655,-0.106057,-0.171560,-0.105056,-0.076026,-0.185853,-0.125826,-0.030235,...,0.880180,919.000000,14.000000,45.00000,20.000000,134.000000,129.000000,0.000000,0.000000,0.000000
25%,-0.048794,-0.061848,-0.036372,-0.009608,-0.122009,-0.040004,-0.020599,-0.107815,-0.044900,0.006553,...,0.890774,1003.500000,28.000000,50.00000,22.000000,242.000000,232.000000,0.000000,0.200000,0.000000
50%,-0.026015,-0.032280,-0.015854,0.018101,-0.087591,-0.023647,0.002887,-0.079946,-0.021781,0.019506,...,0.902423,1054.000000,28.000000,52.50000,24.500000,269.000000,259.000000,1.000000,0.333333,1.000000
75%,-0.009166,-0.009521,0.011781,0.049164,-0.026148,-0.001903,0.031278,-0.055150,0.006157,0.031526,...,0.911330,1097.000000,30.000000,52.50000,24.500000,321.000000,300.000000,1.000000,0.875000,1.000000
max,0.076832,0.049612,0.073470,0.117160,0.088323,0.068677,0.096358,0.016453,0.097132,0.125401,...,0.942958,1266.000000,37.000000,60.00000,34.000000,389.000000,396.000000,1.000000,1.000000,1.000000


Split data into dev and test, and then split that into train and validation.

In [29]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference'])

In [30]:
df = df.drop(columns = ['Jaccard_Score'])


In [31]:
df.columns

Index([                                  0,
                                         1,
                                         2,
                                         3,
                                         4,
                                         5,
                                         6,
                                         7,
                                         8,
                                         9,
       ...
       'scaled_local_symmetric_percent_id',
                         'query_align_len',
                         'query_align_cov',
                       'subject_align_len',
                       'subject_align_cov',
                               'bit_score',
                           'm_protein_len',
                           't_protein_len',
                           'protein_match',
                         'structure_match'],
      dtype='object', length=1292)

In [32]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(56, 1292)
(11, 1292)
(47, 1292)
(9, 1292)


In [33]:
#ID target and features, separate into separate arrays

target = [columns for columns in df[['protein_match', 'structure_match']]]
input_features = [columns for columns in df.drop(columns=['protein_match', 'structure_match'])]

In [34]:
input_features

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [35]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values
test_y = test[target].values

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(56, 1290) (11, 1290) (56, 2) (11, 2)


In [36]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values
val_y = val[target].values

Scale the data

In [37]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [38]:
#Random Forest

model = sklearn.ensemble.RandomForestClassifier()
model.fit(dev_X, dev_y)

RandomForestClassifier()

In [115]:
# Obtain predicted class probabilities
y_pred_prob = model.predict_proba(dev_X)

# Compute uncertainty estimates using out-of-bag samples
oob_predictions = model.oob_decision_function_

uncertainty = np.mean(np.var(oob_predictions, axis=1), axis=0)

# 'y_pred_prob' contains predicted class probabilities, and 'uncertainty' represents the uncertainty estimates


In [116]:
uncertainty

array([0.09550174, 0.09550174])

In [117]:
import joblib

In [118]:
joblib.dump(model, 'trained_model.pkl')

['trained_model.pkl']

In [119]:
joblib.load('trained_model.pkl')

RandomForestClassifier(oob_score=True)

Test the model, report relevant statistics

In [39]:
score = model.score(test_X, test_y)
print('Model score is: {}'.format(score))

preds = model.predict(test_X)
len(preds)

Model score is: 0.5454545454545454


11

In [40]:
preds

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1]])

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = test_y

predicted_labels = preds

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision
precision = precision_score(true_labels, predicted_labels, average=None)

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average=None)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average=None)

# Print the scores
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.5454545454545454
Precision: [0.42857143 0.42857143]
Recall: [0.75 0.75]
F1 Score: [0.54545455 0.54545455]
